In [13]:
# load_data.py
import pandas as pd
import numpy as np
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy

# This is the same data that was plotted for similarity earlier
# with one new user "E" who has rated only movie 1

userNum = 200
a = pd.DataFrame(np.random.random([50, userNum]))
a = (a>0.99)*1-(a<0.01)
a = a+2
a = a.rename(columns={i:'user%a'%i for i in range(userNum)})

from scipy.sparse import coo_matrix
coo = coo_matrix(a)

ratings_dict = {
    "wineID": coo.row,
    "userID": coo.col,
    "rating": coo.data
}

df_rating = pd.DataFrame(ratings_dict)
reader = Reader()

# Loads Pandas dataframe
data = Dataset.load_from_df(df_rating[["userID", "wineID", "rating"]], reader)
trainset, testset = train_test_split(data, test_size=.15)


In [14]:
# load_data.py

from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy

# This is the same data that was plotted for similarity earlier
# with one new user "E" who has rated only movie 1

userNum = 200
a = pd.DataFrame(np.random.random([50, userNum]))
a = (a>0.99)*1-(a<0.01)
a = a+2
a = a.rename(columns={i:'user%a'%i for i in range(userNum)})

from scipy.sparse import coo_matrix
coo = coo_matrix(a)

ratings_dict = {
    "wineID": coo.row,
    "userID": coo.col,
    "rating": coo.data
}

df_rating = pd.DataFrame(ratings_dict)
reader = Reader()

# Loads Pandas dataframe
data = Dataset.load_from_df(df_rating[["userID", "wineID", "rating"]], reader)
trainset, testset = train_test_split(data, test_size=.15)


In [15]:
a.head()

,user0,user1,user2,user3,user4,user5,user6,user7,user8,user9,...,user190,user191,user192,user193,user194,user195,user196,user197,user198,user199
0,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
1,2,2,2,2,2,2,2,2,2,1,...,2,2,2,2,2,2,2,2,2,2
2,2,2,2,2,2,2,2,3,2,2,...,2,2,2,2,2,2,2,2,2,2
3,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
4,2,2,2,2,2,2,2,2,1,2,...,2,2,2,2,2,2,2,2,2,2


In [16]:
df_rating.head()

,wineID,userID,rating
0,0,0,2
1,0,1,2
2,0,2,2
3,0,3,2
4,0,4,2


In [17]:
# ----- SVD ----- #

param_grid = {'n_factors': [120, 160], 
              'n_epochs': [30], 
              'lr_all': [0.001, 0.003, 0.005, 0.008],
              'reg_all': [0.08, 0.1, 0.15]}

In [18]:
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

In [19]:
gs.fit(data)

In [20]:
algo = gs.best_estimator['rmse']

In [21]:
print(gs.best_score['rmse'])

0.14852482171652734


In [22]:
print(gs.best_params['rmse'])

{'n_factors': 120, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.15}


In [23]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1435  0.1367  0.1716  0.1393  0.1495  0.1481  0.0125  
MAE (testset)     0.0388  0.0380  0.0468  0.0374  0.0398  0.0402  0.0034  
Fit time          0.85    0.81    0.80    0.80    0.81    0.81    0.02    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([0.14347682, 0.13672758, 0.17159693, 0.13926479, 0.14945475]),
 'test_mae': array([0.03878051, 0.0379653 , 0.04684195, 0.03735827, 0.0398276 ]),
 'fit_time': (0.8488271236419678,
  0.806643009185791,
  0.8041388988494873,
  0.8028638362884521,
  0.8058040142059326),
 'test_time': (0.014696121215820312,
  0.013734102249145508,
  0.01499485969543457,
  0.014512062072753906,
  0.01416015625)}

In [24]:
# Use the new parameters with the train data
algo = SVD(n_factors=160, n_epochs=30, lr_all=0.008, reg_all=0.08)
algo.fit(trainset)
test_pred = algo.test(testset)
print("SVD : Test Set")
accuracy.rmse(test_pred, verbose=True)

SVD : Test Set
RMSE: 0.1520


0.15197255825459574

In [25]:
prediction = algo.predict(12, 1)
prediction.est

2.0175659699867934